In [1]:
from math import ceil, pi, e
from sympy import series, Symbol, Rational, diff, factorial, fraction, nsimplify, solveset, S, ln, expand
from sympy.functions import sin, cos, tan
import pandas as pd
from IPython.display import display, HTML
from termcolor import colored

In [18]:
def truncate(floatt):
  '''truncate floatt to n digits'''
  return int(round(floatt,10) * 10000)/10000.0

def printt(string, color, end='\n'):
  print(colored(string,color),end=end)

def compare(comparefx, derivvalues, xvals, length):
  print(f'fx\t= {comparefx}')
  cmplist = list()
  for i in range(1,len(xvals)+1):
    string = "'"*i
    deriv = diff(comparefx,x,i)
    cmplist.append(truncate(deriv.subs(x,xvals[i-1])))
    print(f"f{string}x\t= {deriv} {colored(f' => {cmplist[-1]}','blue')}")
  print()
  for i in range(len(xvals)):
    string = "'"*(i+1)
    print(f'|f{string}({xvals[i]}) - P{length}{string}({xvals[i]})|\t= {abs(derivvalues[i] - cmplist[i])}')

def newtondivideddiff(coords, x, xs, derivtimes, xvals, comparefx):
  values = [[i[1] for i in coords]]
  length = len(coords)
  dif = 1
  for i in range(len(coords)-1):
    temp = list()
    orders = values[-1]
    for j in range(len(orders)-1):
      temp.append(truncate((orders[j+1]-orders[j])/(xs[j+dif]-xs[j])))
    dif+=1
    values.append(temp)

  # creating table
  data = {'x':xs,'y':values[0]}
  [data.update({i:values[i]}) for i in range(1,length)]

  df = pd.DataFrame.from_dict(data, orient='index')
  df = df.transpose().fillna('-') # table created
  display(HTML(df.to_html(index=False)))

  # solving Px

  Px = ['y0']
  for i in range(1,length):
    string = 'y[x0'
    for j in range(i):
      string+=f',x{j+1}'
    string+=']'
    for j in range(i):
      string+=f'(x-x{j})'
    Px.append(string)
  print(f'P{length-1}x =\t',end='')
  [print(f'{i} +\n\t',end='') for i in Px]
  print()

  Px = [values[0][0]]
  listt = [values[0][0]]
  for i in range(1,length):
    string = f'({values[i][0]})'
    temp = values[i][0]
    for j in range(i):
      string += f'(x-{xs[j]})'
      temp *= (x-xs[j])
    Px.append(string)
    listt.append(expand(temp))

  print(f'P{length-1}x =\t',end='')
  [print(f'{i} +\n\t',end='') for i in Px]
  print()

  print(f'P{length-1}x =\t',end='')
  [print(f'{i} +\n\t',end='') for i in listt]
  print()
  
  derivvalues = list()
  for i in range(1,derivtimes+1):
    string = "'"*i
    print(f'P{length-1}{string}x =\t',end='')
    temp = [diff(j,x,i) for j in listt]
    [print(f'{j} +\n\t',end='') for j in temp]

    print(f'\nP{length-1}{string}x =\t',end='')
    [print(f"{str(j).replace('x',f'({xvals[i-1]})')} +\n\t",end='') for j in temp]

    derivs = sum(temp)
    printt(f'\nP{length-1}{string}x =\t{derivs}','blue')
    if xvals!=None:
      derivvalues.append(truncate(derivs.subs(x,xvals[i-1])))
      printt(f'P{length-1}{string}x =\t{derivvalues[-1]}','blue')
    print()

  if comparefx!=None:
    compare(comparefx, derivvalues, xvals, length-1)
  




In [19]:
x = Symbol('x')

coords = [(1,0.6767),(1.5,0.3734),(1.6,0.3261),(2.5,0.0842)]
xs = [i[0] for i in coords]
derivtimes = 2
xvals = [1.5, 1.7] # [value for 1st deriv, value for 2nd deriv, ...]
comparefx = 5*x*e**(-2*x) # if not given, change value to None

newtondivideddiff(coords, x, xs, derivtimes, xvals, comparefx)

x,y,1,2,3
1.0,0.6767,-0.6066,0.2226,-0.0122
1.5,0.3734,-0.473,0.2043,-
1.6,0.3261,-0.2687,-,-
2.5,0.0842,-,-,-


P3x =	y0 +
	y[x0,x1](x-x0) +
	y[x0,x1,x2](x-x0)(x-x1) +
	y[x0,x1,x2,x3](x-x0)(x-x1)(x-x2) +
	
P3x =	0.6767 +
	(-0.6066)(x-1) +
	(0.2226)(x-1)(x-1.5) +
	(-0.0122)(x-1)(x-1.5)(x-1.6) +
	
P3x =	0.6767 +
	-0.6066*x + 0.6066 +
	0.2226*x**2 - 0.5565*x + 0.3339 +
	-0.0122*x**3 + 0.05002*x**2 - 0.0671*x + 0.02928 +
	
P3'x =	0 +
	-0.606600000000000 +
	0.4452*x - 0.5565 +
	-0.0366*x**2 + 0.10004*x - 0.0671 +
	
P3'x =	0 +
	-0.606600000000000 +
	0.4452*(1.5) - 0.5565 +
	-0.0366*(1.5)**2 + 0.10004*(1.5) - 0.0671 +
	
P3'x =	-0.0366*x**2 + 0.54524*x - 1.2302
P3'x =	-0.4946

P3''x =	0 +
	0 +
	0.445200000000000 +
	-0.0732*x + 0.10004 +
	
P3''x =	0 +
	0 +
	0.445200000000000 +
	-0.0732*(1.7) + 0.10004 +
	
P3''x =	-0.0732*x + 0.54524
P3''x =	0.4208

fx	= 5*2.71828182845905**(-2*x)*x
f'x	= -10.0*2.71828182845905**(-2*x)*x + 5*2.71828182845905**(-2*x)  => -0.4978
f''x	= 20.0*2.71828182845905**(-2*x)*(x - 1)  => 0.4672

|f'(1.5) - P3'(1.5)|	= 0.003200000000000036
|f''(1.7) - P3''(1.7)|	= 0.0464
